In [ ]:
!pip3 install wordcloud
!pip3 install tqdm
!pip3 install pyLDAvis

In [ ]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import nltk
import sklearn
nltk.download('stopwords')
nltk.download('punkt')
from pyLDAvis import sklearn
import pyLDAvis
from wordcloud import WordCloud

In [ ]:
STOPWORDS = set(nltk.corpus.stopwords.words('english'))

In [ ]:
df = pd.read_csv("NOTEEVENTS.csv.gz")

In [ ]:
def whole_dataset(func):
    def inner():
        ret = []
        chunks=pd.read_csv("NOTEEVENTS.csv.gz",chunksize=100000)
        for chunk in tqdm(chunks):
            chunk.columns = [c.lower() for c in chunk.columns]
            ret.append(func(chunk))
        return ret
    return inner

In [ ]:
@whole_dataset
def check_na(chunk):
    return chunk.text.isna().any()

In [ ]:
check_na()

In [ ]:
words=defaultdict(int)

In [ ]:
def count(text):
    w = text.split()
    for word in w:
        if word not in STOPWORDS:
            words[word.lower()]+=1

In [ ]:
@whole_dataset
def get_word_counts(chunk):
    chunk.text.apply(lambda x:count(x))

In [ ]:
get_word_counts()

In [ ]:
len(words)

In [ ]:
words

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
wc = WordCloud(width=800, height=600,background_color="white", max_words=1000)
# generate word cloud
wc.generate_from_frequencies(words)

# show
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
sections =defaultdict(int)

In [ ]:
import re

In [ ]:
def section_count(text):
    matches = [i[0] for i in  re.findall('\n((.?)*):\n',text)]
    for match in matches:
        sections[match.lower().strip()]+=1

In [ ]:
@whole_dataset
def get_section_counts(chunk):
    chunk.text.apply(lambda x:section_count(x))

In [ ]:
get_section_counts()

In [ ]:
sections['assessment']

In [ ]:
sorted(sections, key = lambda x: sections[x], reverse=True)

In [ ]:
des= sorted(sections, key = lambda x: sections[x], reverse=True)

In [ ]:
des =[s.strip() for s in des ]

In [ ]:
sample =des[:30]
sample_count = [sections[i] for i in sample]

In [ ]:
plt.barh(sample,sample_count)
plt.xticks(rotation=90)
plt.show()

In [ ]:
all

In [ ]:
nltk.tokenize.sent_tokenize(chunk.TEXT[6])

In [ ]:
tf_vectorizer.fit(chunkgen())

In [ ]:
x = chunkgen()

In [ ]:
next(x)

In [ ]:
def chunkgen():
    chunks=pd.read_csv("NOTEEVENTS.csv.gz",chunksize=500000)
    for chunk in tqdm(chunks):
        chunk.columns = [c.lower() for c in chunk.columns]
        text  = chunk.text.to_numpy()
        del chunk
        for i in text:
            yield i

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, PCA
import tensorflow as tf

In [ ]:
nbr_features = 100

# LDA uses raw term frequencies
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=nbr_features, stop_words='english')
term_freqs = tf_vectorizer.fit_transform(df.TEXT)
tf_feature_names = tf_vectorizer.get_feature_names()

# Plot pretty LDA output


In [ ]:
term_freqs

In [ ]:
nbr_topics = 20
lda = LatentDirichletAllocation(n_components=nbr_topics,verbose=1, max_iter=5, learning_offset=50.,random_state=3).fit(term_freqs)
lda_topics = lda.transform(term_freqs)

In [ ]:
lda_vis_data = pyLDAvis.sklearn.prepare(lda, term_freqs, tf_vectorizer)
lda_vis_data_html = pyLDAvis.prepared_data_to_html(lda_vis_data)
pyLDAvis.display(lda_vis_data)